In [2]:
#importing libraries
import numpy as np
import pandas as pd
import random
from math import exp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define triangular membership function with optimized parameters
def triangular_mf(x, a, b, c, ki, wi):
    a = (a + ki) - wi
    b = (b + ki) - wi
    c = (c + ki) - wi
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

# Fuzzy rule evaluation with optimized parameters
def evaluate_rules(features, params):
    Glucose, INS, BMI, DPF, Age = features
    ki, wi = params  # or ki = params[0], wi = params[1] if params is an array
    
    # Membership functions for each feature, adjusted based on ki and wi
    Ageyoung = triangular_mf(Age, 0, 30, 40, ki, wi)
    
    Glucoselow = triangular_mf(Glucose, 0, 85, 100, ki, wi)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160, ki, wi)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200, ki, wi)
    
    INSlow = triangular_mf(INS, 0, 100, 200, ki, wi)
    INSmedium = triangular_mf(INS, 100, 200, 400, ki, wi)
    INShigh = triangular_mf(INS, 200, 400, 600, ki, wi)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30, ki, wi)
    BMImedium = triangular_mf(BMI, 20, 30, 40, ki, wi)
    BMIhigh = triangular_mf(BMI, 30, 40, 50, ki, wi)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6, ki, wi)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0, ki, wi)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0, ki, wi)
    
    # Fuzzy rules
    DMverylow = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    DMverylow = max(Glucoselow, INSlow, BMImedium, DPFlow, Ageyoung)

    # Aggregate into a single value for DM
    DM = max(DMverylow, DMlow, DMmedium, DMhigh)
    
    return DM

def simulated_annealing(fitness_func, initial_temp, final_temp, cooling_rate, max_iterations):
    # Initialize with random values for ki and wi
    current_solution = np.random.uniform(0, 100, 2)
    current_fitness = fitness_func(current_solution)
    
    temperature = initial_temp
    
    while temperature > final_temp:
        for _ in range(max_iterations):
            # Perturb the current solution to get a new one
            new_solution = current_solution + np.random.uniform(-5, 5, 2)
            new_fitness = fitness_func(new_solution)
            
            # If the new solution is better, accept it
            if new_fitness < current_fitness:
                current_solution = new_solution
                current_fitness = new_fitness
            else:
                # Otherwise, accept it based on the Metropolis criterion
                delta = new_fitness - current_fitness
                if random.random() < exp(-delta / temperature):
                    current_solution = new_solution
                    current_fitness = new_fitness
        
        # Cool down
        temperature *= cooling_rate

    return current_solution

# Use Simulated Annealing to find best parameters
best_params = simulated_annealing(fitness, initial_temp=1000, final_temp=1, cooling_rate=0.95, max_iterations=100)
print("Best Parameters ki and wi: ", best_params)

# Test the classifier on the test set using optimized parameters
fuzzy_results_test = [evaluate_rules(features, best_params) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy based on optimized parameters: {acc}")
print(f"Sensitivity based on optimized parameters: {sensitivity}")
print(f"Specificity based on optimized parameters: {specificity}")
